In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
v0_finders = ["Offline", "On-the-fly", "Custom", "Kalman"]
# v0_finders = ["custom"]
observables = ["N(S)/N(S+B)", "N(S)/N(findableS)"]
observables_title = ["Purity", "Efficiency"]
particles = ["AntiLambda", "KaonZeroShort"]
hist_titles = [r"Found $\bar{\Lambda}\rightarrow\bar{p}\pi^{+}$", r"Found $K_S^0\rightarrow\pi^{-}\pi^{+}$"]
edges = [0., 1., 2., 3., 4.]
bin_centers = [0.5, 1.5, 2.5, 3.5]

# init dicts

n_entries = {}
n_entries["N(S)/N(S+B)"] = {}
n_entries["N(S)/N(S+B)"]["AntiLambda"] = []
n_entries["N(S)/N(S+B)"]["KaonZeroShort"] = []
n_entries["N(S)/N(findableS)"] = {}
n_entries["N(S)/N(findableS)"]["AntiLambda"] = []
n_entries["N(S)/N(findableS)"]["KaonZeroShort"] = []

error_entries = {}
error_entries["N(S)/N(S+B)"] = {}
error_entries["N(S)/N(S+B)"]["AntiLambda"] = []
error_entries["N(S)/N(S+B)"]["KaonZeroShort"] = []
error_entries["N(S)/N(findableS)"] = {}
error_entries["N(S)/N(findableS)"]["AntiLambda"] = []
error_entries["N(S)/N(findableS)"]["KaonZeroShort"] = []

for v0_finder in v0_finders:

    filename = f"../sexaquark/test_sig/AnalysisResults_A1.8_{v0_finder.lower()}V0s.root"
    file = uproot.open(filename)
    histograms = file["Hists/"]

    hist_dict = {hist.name: hist for hist in histograms}

    for particle_name in particles:

        hist_name_all = f"Found_All_{particle_name}_Mass"
        values_all, edges_all = hist_dict[hist_name_all].to_numpy()

        hist_name_signal = f"Found_Signal_{particle_name}_Mass"
        values_signal, edges_signal = hist_dict[hist_name_signal].to_numpy()

        hist_name_findable_signal = f"Findable_Signal_{particle_name}_Radius"
        values_findable_signal, edges_findable_signal = hist_dict[hist_name_findable_signal].to_numpy()

        ##

        N_S = np.sum(values_signal)
        N_All =np.sum(values_all)
        N_FindableS = np.sum(values_findable_signal)

        Ratio_NS_NAll = N_S / N_All
        Ratio_NS_NFindableS = N_S / N_FindableS

        n_entries["N(S)/N(S+B)"][particle_name].append(Ratio_NS_NAll)
        n_entries["N(S)/N(findableS)"][particle_name].append(Ratio_NS_NFindableS)

        ##

        Error_Ratio_NS_NAll = np.sqrt(Ratio_NS_NAll * (1 - Ratio_NS_NAll) / N_All)
        Error_Ratio_NS_NFindableS = np.sqrt(Ratio_NS_NFindableS * (1 - Ratio_NS_NFindableS) / N_FindableS)

        error_entries["N(S)/N(S+B)"][particle_name].append(Error_Ratio_NS_NAll)
        error_entries["N(S)/N(findableS)"][particle_name].append(Error_Ratio_NS_NFindableS)

# make four plots

plt.figure(figsize=(10, 10))
counter=0
for title, obs in zip(observables_title, observables):
    for pp, particle_name, hist_title in zip(range(len(particles)), particles, hist_titles):
        counter += 1

        ax = plt.subplot( len(observables) , len(particles), counter)
        ax.errorbar(bin_centers, n_entries[obs][particle_name], xerr=0.5,fmt='none', yerr=error_entries[obs][particle_name], color=colors[pp+2], label=hist_title, linewidth=2)

        ax.set_ylim(0., 1.)
        ax.set_ylabel(title, fontweight="bold", loc='top', fontsize=12)
        ax.set_xticks([i + 0.5 for i in edges[:-1]], v0_finders, fontweight="bold", fontsize=12)

        ax.tick_params(axis='x', which='major', pad=6)
        ax.tick_params(direction='in', top=True, right=True)
        ax.grid(True, linestyle='--', alpha=0.5)

        ax.legend(loc='upper left', framealpha=1, fontsize=12)

plt.tight_layout()
plt.savefig('V0sPerformance.svg')
plt.show()